In [1]:
import ptan
import torch
from torch import nn
import numpy as np

/home/sannhtet/Desktop/Computer Science and AI/AI/Deep-Reinforcement-Learning-Hands-On/.venv/lib/python3.11/site-packages/ignite/handlers/checkpoint.py:16: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


In [2]:
class DQNNet(nn.Module):
    def __init__(self, actions: int):
        super(DQNNet, self).__init__()
        self.actions = actions

    def forward(self, x):
        # we always produce diagonal tensor of shape
        # (batch_size, actions)
        return torch.eye(x.size()[0], self.actions)

In [3]:
net = DQNNet(actions=3)
net(torch.zeros(2, 10))

tensor([[1., 0., 0.],
        [0., 1., 0.]])

In [4]:
selector = ptan.actions.ArgmaxActionSelector()
agent = ptan.agent.DQNAgent(model=net, action_selector=selector)
agent(torch.zeros(2, 5))

(array([0, 1]), [None, None])

In [5]:
selector = ptan.actions.EpsilonGreedyActionSelector(epsilon=1.0)
agent = ptan.agent.DQNAgent(model=net, action_selector=selector)
agent(torch.zeros(10, 5))[0]

array([2, 2, 2, 0, 1, 2, 0, 1, 0, 1])

In [6]:
selector.epsilon = 0.5
agent(torch.zeros(10, 5))[0]

array([0, 1, 2, 0, 1, 0, 1, 1, 0, 0])

In [8]:
selector.epsilon = 0.1
agent(torch.zeros(10, 5))[0]

array([0, 1, 2, 0, 0, 0, 0, 0, 0, 0])

## PolicyAgent

In [9]:
class PolicyNet(nn.Module):
    def __init__(self, actions: int):
        super(PolicyNet, self).__init__()
        self.actions = actions

    def forward(self, x):
        # Now we produce the tensor with first two actions
        # having the same logit scores
        shape = (x.size()[0], self.actions)
        res = torch.zeros(shape, dtype=torch.float32)
        res[:, 0] = 1
        res[:, 1] = 1
        return res

In [10]:
net = PolicyNet(actions=5)
net(torch.zeros(6, 10))

tensor([[1., 1., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 0., 0., 0.]])

In [11]:
selector = ptan.actions.ProbabilityActionSelector()
agent = ptan.agent.PolicyAgent(model=net, action_selector=selector, apply_softmax=True)
agent(torch.zeros(6, 5))[0]

array([0, 1, 1, 1, 0, 0])

In [13]:
torch.nn.functional.softmax(torch.tensor([1., 1., 0., 0., 0.]))

/tmp/ipykernel_12494/120467620.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  torch.nn.functional.softmax(torch.tensor([1., 1., 0., 0., 0.]))


tensor([0.3222, 0.3222, 0.1185, 0.1185, 0.1185])